## Create FHIR STU3 CapStatement Resource

### Outline:

- Source excel with requirements
- pandas to convert in python Ordered Dict
- build json
- generate narrative using Jinja2 templates

### Prerequisites:

- Python 3.6 or greater

### Import FHIRClient and other libraries

In [401]:
%config IPCompleter.greedy=True

In [402]:
from fhirclient.models import searchparameter as SP
from fhirclient.models import capabilitystatement as CS
from fhirclient.models import bundle as B
import fhirclient.models.identifier as I
import fhirclient.models.coding as C
import fhirclient.models.codeableconcept as CC
import fhirclient.models.fhirdate as D
import fhirclient.models.extension as X
import fhirclient.models.contactdetail as CD
import fhirclient.models.fhirreference as FR
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from IPython import display as Display
from pprint import pprint
from stringcase import snakecase, titlecase
from collections import namedtuple
from pandas import *
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape

#### Inspect ElementProperties as reference

In [403]:
CS.CapabilityStatementRestResource().elementProperties()

[('extension',
  'extension',
  fhirclient.models.extension.Extension,
  True,
  None,
  False),
 ('id', 'id', str, False, None, False),
 ('modifierExtension',
  'modifierExtension',
  fhirclient.models.extension.Extension,
  True,
  None,
  False),
 ('conditionalCreate', 'conditionalCreate', bool, False, None, False),
 ('conditionalDelete', 'conditionalDelete', str, False, None, False),
 ('conditionalRead', 'conditionalRead', str, False, None, False),
 ('conditionalUpdate', 'conditionalUpdate', bool, False, None, False),
 ('documentation', 'documentation', str, False, None, False),
 ('interaction',
  'interaction',
  fhirclient.models.capabilitystatement.CapabilityStatementRestResourceInteraction,
  True,
  None,
  True),
 ('profile',
  'profile',
  fhirclient.models.fhirreference.FHIRReference,
  False,
  None,
  False),
 ('readHistory', 'readHistory', bool, False, None, False),
 ('referencePolicy', 'referencePolicy', str, True, None, False),
 ('searchInclude', 'searchInclude', str, 

####  Assign Global Variables


Here is where we assign all the global variables for this example such as the local paths for file input and output

In [404]:
md_templates = ['templ_1.j2']

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

fhir_term_server = 'http://test.fhir.org/r3'
fhir_test_server = 'http://test.fhir.org/r3'

headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}

fhir_base_url = 'http://hl7.org/fhir/'

base_id = "argo-q"

canon = "http://fhir.org/guides/argonaut-questionnaire/"

pre = "Argonaut"

publisher = 'The Argonaut Project'

publisher_endpoint = dict(
                        system = 'url',
                        value = 'https://github.com/argonautproject/questionnaire/issues'
                        )

f_jurisdiction =  CC.CodeableConcept({
      "coding" : [
        {
          "system" : "urn:iso:std:iso:3166",
          "code" : "US"
        }
      ]
    })

conf_url = 'http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation'

none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False']

sep_list = (',', ';', ' ', ', ', '; ')

f_now = D.FHIRDate(f'{datetime.now().isoformat()}Z')

#### Conformance Extension

In [405]:
def get_conf(conf='MAY'):
    return [X.Extension(dict(
        url = conf_url,
        valueCode = conf
        ))]

### validate

In [406]:
# *********************** validate Resource ********************************

def validate(r):
    fhir_test_server = 'http://fhirtest.uhn.ca/baseDstu3'
    #fhir_test_server = 'http://test.fhir.org/r3'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
      # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
        )
    
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = post(f'{fhir_test_server}/Questionnaire/$validate', params = params, headers = headers, data = dumps(r.as_json()))
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Write to File

In [407]:
 def write_file(name, data): # write file
    # out_path = ''
    out_path = '//ERICS-AIR-2/ehaas/Documents/FHIR/Argo-Questionnaire/source/resources/'
    with open(f'{Path(out_path)}/{name}.json', 'w') as f:
        f.write(data)

### Get Search Parameter input data

#### first the meta sheet

In [408]:
#in_path = Path('/Users/ehaas/Documents/FHIR/pyfhir/test/')
in_path =''
#out_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
out_path=''
#out_path = Path("C:/Users/Eric/Documents/Jan_2019_FHIR_Experience")

in_file = ['AssBank']

xls = ExcelFile(f'{in_path}{in_file[0]}.xlsx')
df = read_excel(xls,'meta',na_filter = False)


df

,Element,Value
0,id,assessmentbank
1,description,This section outlines conformance requirements...
2,ig,http://fhir.org/guides/argonaut-questionnaire/...
3,mode,server
4,documentation,The Argonaut questionnaire Assessment-Bank **S...
5,security,For general security consideration refer to th...


#### Create NamedTuple from df to use dot notation

In [409]:
d = dict(zip(df.Element, df.Value))
meta = namedtuple("Meta", d.keys())(*d.values())      
         
meta.id

'assessmentbank'

### Create CS instance

In [410]:
cs = CS.CapabilityStatement()
cs.id = meta.id
cs.url = f'{canon}{meta.id}'
cs.version = '0.0.0'  # placeholder changed by build
cs.name = snakecase(meta.id)
cs.title = f'{pre} {titlecase(meta.id)} {cs.resource_type}'
cs.status = 'active'
cs.experimental = False
cs.date = f_now  # as FHIRDate
cs.publisher = publisher
cs.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
cs.description = meta.description
cs.jurisdiction = [f_jurisdiction]
cs.kind = 'requirements'
cs.fhirVersion = '3.0.1'
cs.acceptUnknown = 'both'
cs.format = [
    "application/fhir+xml",
    "application/fhir+json"
  ]
cs.patchFormat = [
    "application/json-patch+json",
  ]
cs.implementationGuide = meta.ig.split(",")
rest = CS.CapabilityStatementRest(dict(
    mode = meta.mode,
    documentation = meta.documentation,
    security = dict(
        description = meta.security
        )
    ))
cs.rest = [rest]




cs.as_json()

{'id': 'assessmentbank',
 'acceptUnknown': 'both',
 'contact': [{'telecom': [{'system': 'url',
     'value': 'https://github.com/argonautproject/questionnaire/issues'}]}],
 'date': '2019-02-19T17:31:53.088515Z',
 'description': 'This section outlines conformance requirements for the Argonaut Questionnaire Assessment-Bank Server which is responsible for storing Questionnaires and providing responses to the requests submitted by the  Form Author/Editor and Provider EHRs. The Argonaut Questionnaire Profile and the various interactions outlined in this guide are the RESTful artifacts and interactions that need to be supported. Note that the Argonaut Profiles identify the structural constraints, terminology bindings and invariants and the individual Argonaut SearchParameter resources define the definitions, comparators, modifiers and usage constraints.',
 'experimental': False,
 'fhirVersion': '3.0.1',
 'format': ['application/fhir+xml', 'application/fhir+json'],
 'implementationGuide': ['h

#### Then the list of IG profiles ( for STU3 )

In [411]:
xls = ExcelFile(f'{in_path}{in_file[0]}.xlsx')
df = read_excel(xls,'profiles',na_filter = False)

cs.profile = [] 
for p in df.itertuples(index=True):
    print(p.Profile, p.Conformance)
    ref = FR.FHIRReference(dict(reference = p.Profile))
    ref.extension = get_conf(p.Conformance)
    cs.profile.append(ref)
                                           
    
cs.as_json()

http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/argo-questionnaire-valueset SHALL
http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/argo-questionnaire SHALL
http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/argo-questionnaireresponse SHALL
http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-multipleAnd SHALL
http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-multipleOr SHALL
http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-itemOrder SHALL
http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-responsePeriod SHALL
http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-timeLimit SHALL


{'id': 'assessmentbank',
 'acceptUnknown': 'both',
 'contact': [{'telecom': [{'system': 'url',
     'value': 'https://github.com/argonautproject/questionnaire/issues'}]}],
 'date': '2019-02-19T17:31:53.088515Z',
 'description': 'This section outlines conformance requirements for the Argonaut Questionnaire Assessment-Bank Server which is responsible for storing Questionnaires and providing responses to the requests submitted by the  Form Author/Editor and Provider EHRs. The Argonaut Questionnaire Profile and the various interactions outlined in this guide are the RESTful artifacts and interactions that need to be supported. Note that the Argonaut Profiles identify the structural constraints, terminology bindings and invariants and the individual Argonaut SearchParameter resources define the definitions, comparators, modifiers and usage constraints.',
 'experimental': False,
 'fhirVersion': '3.0.1',
 'format': ['application/fhir+xml', 'application/fhir+json'],
 'implementationGuide': ['h


#### add Resources

In [412]:
df = read_excel(xls,'resources',na_filter = False)
df_i = read_excel(xls,'interactions',na_filter = False)
df_sp = read_excel(xls,'sps',na_filter = False)

def get_i():
    int_list = []
    for i in df_i.itertuples(index=True):
        # print(i.code, i.conf)
        int  = CS.CapabilityStatementRestResourceInteraction()
        int.code = i.code
        int.extension = get_conf(i.conf)    
        int_list.append(int.as_json())
    return int_list


def get_sp(r_type):
    sp_list = []
    for i in df_sp.itertuples(index=True):
        if i.Resource == r_type:
            # print(i.Parameter, i.Resource, i.Conformance)
            sp  = CS.CapabilityStatementRestResourceSearchParam()
            sp.name = i.Parameter
            sp.definition = (f'{canon}SearchParameter/{i.Resource}-{i.Parameter}' if i.Update == 'Y' or i.Exists =='N'
                             else f'{fhir_base_url}SearchParameter/{i.Base}-{i.Parameter}')
            print(sp.definition)
            sp.type = i.Type
            sp.extension = get_conf(i.Conformance)    
            sp_list.append(sp.as_json())
    return sp_list


rest.resource =  []
for r in df.itertuples(index=True):
    # print(r.type, r.conformance, r.readHistory)
    res = CS.CapabilityStatementRestResource(
    dict(
        type = r.type,
        documentation = r.documentation,
        versioning = r.versioning,
        readHistory = None if r.readHistory is None else r.readHistory == 'True',
        updateCreate = None if r.readHistory is None else r.readHistory == 'True',
        referencePolicy = r.referencePolicy.split(","),
        interaction = get_i(),
        searchParam = get_sp(r.type)
        
        ) 
    )
    res.extension = get_conf(r.conformance)
    
    rest.resource.append(res)
cs.rest = [rest]
    
print(dumps(cs.as_json(),indent=3))

http://hl7.org/fhir/SearchParameter/Resource-_id
http://hl7.org/fhir/SearchParameter/Questionnaire-url
http://fhir.org/guides/argonaut-questionnaire/SearchParameter/Questionnaire-status
http://fhir.org/guides/argonaut-questionnaire/SearchParameter/Questionnaire-title
http://fhir.org/guides/argonaut-questionnaire/SearchParameter/Questionnaire-publisher
http://fhir.org/guides/argonaut-questionnaire/SearchParameter/Questionnaire-version
http://fhir.org/guides/argonaut-questionnaire/SearchParameter/Questionnaire-context-type-value
{
   "id": "assessmentbank",
   "acceptUnknown": "both",
   "contact": [
      {
         "telecom": [
            {
               "system": "url",
               "value": "https://github.com/argonautproject/questionnaire/issues"
            }
         ]
      }
   ],
   "date": "2019-02-19T17:31:53.088515Z",
   "description": "This section outlines conformance requirements for the Argonaut Questionnaire Assessment-Bank Server which is responsible for storing Qu

### Validate

In [413]:
 #validate and write to file

print('...validating')
r = validate(cs)
display(Display.HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))



...validating


WARNING,[CapabilityStatement.jurisdiction],"None of the codes provided are in the value set http://hl7.org/fhir/ValueSet/jurisdiction (http://hl7.org/fhir/ValueSet/jurisdiction, and a code should come from this value set unless it has no suitable code) (codes = urn:iso:std:iso:3166#US)"


### Write to folder

In [414]:
# save to file

rjson = dumps(cs.as_json(), indent=3)
name =f'capabilitystatement-{cs.id.lower()}'
print(name)
write_file(name, rjson)

capabilitystatement-assessmentbank


### Display SP resources

### Create Markdown Text for Search.

- Using Jinja2 Template create markdown file for search section in profiles profiles

In [567]:
r_type = ['Questionnaire', 'QuestionnaireResponse', 'Patient']  # TODO  loop over all types in sp_list

template = env.get_template(md_template[0])
for t in r_type:
    sp_bytype = [i for i in sp_list if i.base[0] == r_type]
    display(Markdown(template.render(sp_bytype=sp_bytype,search_type=search_type)))
    search_md = template.render(sp_bytype=sp_bytype,search_type=search_type)

with open(f'{out_path}{r_type}-search.md','w') as f:
    f.write(search_md)



### Quick Start
Below is an overview of the required set of RESTful FHIR interactions - for
example, search and read operations - for this profile. See the [Conformance requirements]
for a complete list of supported RESTful interactions for this IG.

#### Supported Searches

{% include link-list.md %}

### Quick Start
Below is an overview of the required set of RESTful FHIR interactions - for
example, search and read operations - for this profile. See the [Conformance requirements]
for a complete list of supported RESTful interactions for this IG.

#### Supported Searches

{% include link-list.md %}

### Quick Start
Below is an overview of the required set of RESTful FHIR interactions - for
example, search and read operations - for this profile. See the [Conformance requirements]
for a complete list of supported RESTful interactions for this IG.

#### Supported Searches

{% include link-list.md %}

### Create Markdown Text for SearchParameters Page

- Using Jinja2 Template create markdown file for searchparameters page

In [571]:
# get types:
r_list = []
for i in sp_list:
    for j in i.base:
        r_list.append(j)

r_list = sorted(set(r_list))

# md_template = 'sp_list_page.j2'

template = env.get_template(md_template[1])
display(Markdown(template.render(sp_list=sp_list,r_list=r_list)))
searchparameters_md = template.render(sp_list=sp_list,r_list=r_list)
with open(f'{out_path}searchparameters.md','w') as f:
    f.write(searchparameters_md)

---
title: Operations and Search Parameters
layout: default
topofpage: true
sectionnumbering: true
---

The following search parameters have been defined for the {{site.data.fhir.igName}} Implementation Guide.  For more information on the [FHIR RESTful search api]and the standard [Search Param Registry] see the FHIR specification.

<!-- Operations


  { % include list-simple-operationdefinitions.xhtml % }


-->

Search Parameter


**HealthcareService**
  - [active](SearchParameter-HealthcareService-active.html)

**Patient**

**Person**

**Questionnaire**
  - [url](SearchParameter-Questionnaire-url.html)
  - [status](SearchParameter-Questionnaire-status.html)
  - [title](SearchParameter-Questionnaire-title.html)
  - [publisher](SearchParameter-Questionnaire-publisher.html)
  - [version](SearchParameter-Questionnaire-version.html)

**QuestionnaireResponse**
  - [questionnaire](SearchParameter-QuestionnaireResponse-questionnaire.html)
  - [patient](SearchParameter-QuestionnaireResponse-patient.html)
  - [context](SearchParameter-QuestionnaireResponse-context.html)
  - [status](SearchParameter-QuestionnaireResponse-status.html)
  - [author](SearchParameter-QuestionnaireResponse-author.html)
  - [source](SearchParameter-QuestionnaireResponse-source.html)

**RelatedPerson**
  - [birthdate](SearchParameter-individual-birthdate.html)

{% include link_list.md %}

### Update CapabilityStatement for Search
- instantiate new or existing CapStatement as fhir model
- add combination extensions
- add markdown descriptions from above

In [ ]:
# instantiate new or existing CapStatement:

def interaction(code):
    return CS.CapabilityStatementRestResourceInteraction({'code' : code})

def search_cs(sp):
    template = env.get_template(md_template[2])
    return CS.CapabilityStatementRestResourceSearchParam(
    dict(
    name = sp.code,
    definition = sp.url,
    type = sp.type,
    documentation = template.render(sp=sp, search_type=search_type)
    )
    )

cs = CS.CapabilityStatement(dict(
                            date = now,
                            status = 'active',
                            acceptUnknown = 'both',
                            format = ['json','xml'],
                            fhirVersion = '3.0.1',
                            kind = 'requirements'     
                            ))



cs.rest = [CS.CapabilityStatementRest()]  
cs.rest[0].mode = 'server'
cs.rest[0].resource = []



print(r_list)    
for r in r_list:
    rest_resource = CS.CapabilityStatementRestResource()
    rest_resource.type = r
    rest_resource.interaction = [interaction(code) for code in ['read','vread', 'history-instance', 'search-type']]
    cs.rest[0].resource.append(rest_resource)
    rest_resource.searchParam = [search_cs(sp=sp) for sp in sp_list if sp.base[0] == r]
print(dumps(cs.as_json(),indent = 3))
    
    



### TODO

- Save SearchParameters and CapabilityStatements.
- Cycle through the quick starts
- Update the CSV files